# Drone Trajectory Planner

In this project, we will develop the drone trajectory planner. This notebook serves as the main file for the project, where we will refer to the instructions and demonstrate our code.

Please follow week by week instructions, which includes writing the code in the `src/` folder.

In [1]:
# Import all the files and libraries required for the project
%load_ext autoreload
%autoreload 2
import copy
    
import numpy as np

from src.camera_utils import compute_image_footprint_on_surface, compute_ground_sampling_distance, project_world_point_to_image, reproject_image_point_to_world
from src.data_model import Camera, DatasetSpec
from src.plan_computation import compute_distance_between_images, compute_speed_during_photo_capture, generate_photo_plan_on_grid
from src.visualization import plot_photo_plan

# Week 1: Introduction

No code contribution expected this week

# Week 2: Camera System Modeling and Operations

We plan to
- Model the simple pinhole camera system
- Write utility functions to
    - project a 3D world point to an image
    - Compute image footprint on a surface
    - Compute the Ground Sampling Distance

## Model the camera parameters

We want to model the following camera parameters in Python:
- focal length along x axis (in pixels)
- focal length along y axis (in pixels)
- optical center of the image along the x axis (in pixels)
- optical center of the image along the y axis (in pixels)
- Size of the sensor along the x axis (in mm)
- Size of the sensor along the y axis (in mm)
- Number of pixels in the image along the x axis
- Number of pixels in the image along the y axis

I recommend to use `dataclasses` ([Python documentation](https://docs.python.org/3/library/dataclasses.html), [Blog](https://www.dataquest.io/blog/how-to-use-python-data-classes/) to model these parameters.

$\color{red}{\text{TODO: }}$ Implement `Camera` in `src/data_model.py`

In [2]:
# Define the parameters for Skydio VT300L - Wide camera
# Ref: https://support.skydio.com/hc/en-us/articles/20866347470491-Skydio-X10-camera-and-metadata-overview
fx = 4938.56
fy = 4936.49
cx = 4095.5
cy = 3071.5
sensor_size_x_mm = 13.107 # single pixel size * number of pixels in X dimension
sensor_size_y_mm = 9.830 # single pixel size * number of pixels in Y dimension
image_size_x = 8192
image_size_y = 6144

camera_x10 = Camera(fx, fy, cx, cy, sensor_size_x_mm, sensor_size_y_mm, image_size_x, image_size_y)

In [3]:
print(f"X10 camera model: {camera_x10}")

X10 camera model: Camera(fx=4938.56, fy=4936.49, cx=4095.5, cy=3071.5, sensor_size_x_mm=13.107, sensor_size_y_mm=9.83, image_size_x_px=8192, image_size_y_px=6144)


## Project 3D world points into the image


![Camera Projection](assets/image_projection.png)
Reference: [Robert Collins CSE483](https://www.cse.psu.edu/~rtc12/CSE486/lecture12.pdf)


Equations to implement:
$$ x = f_x \frac{X}{Z} $$
$$ y = f_y \frac{Y}{Z} $$
$$ u = x + c_x $$
$$ v = y + c_y $$

$\color{red}{\text{TODO: }}$ Implement function `project_world_point_to_image` in `src/camera_utils.py`

In [4]:
point_3d = np.array([25, -30, 50], dtype=np.float32)
expected_uv = np.array([6564.80, 109.60], dtype=np.float32)
uv = project_world_point_to_image(camera_x10, point_3d)

print(f"{point_3d} projected to {uv}")

assert np.allclose(uv, expected_uv, atol=1e-2)

[ 25. -30.  50.] projected to [6564.7803   109.60571]


## Compute Image Footprint on the surface

We have written code to *project* a 3D point into the image. The reverse operation is reprojection, where we take $(x, y)$ and compute the $(X, Y)$ for a given value of $Z$. Note that while going from 3D to 2D, the depth becomes ambiguous so we need the to specify the $Z$.

An image's footprint is the area on the surface which is captured by the image. We can take the two corners of the image and reproject them at a given distance to obtain the width and length of the image.

$\color{red}{\text{TODO: }}$ Implement function `compute_image_footprint_on_surface` in `src/camera_utils.py`

In [5]:
footprint_at_100m = compute_image_footprint_on_surface(camera_x10, 100)
expected_footprint_at_100m = np.array([165.88, 124.46], dtype=np.float32)

print(f"Footprint at 100m = {footprint_at_100m}")

assert np.allclose(footprint_at_100m, expected_footprint_at_100m, atol=1e-2)


Footprint at 100m = [165.87831 124.4609 ]


In [6]:
footprint_at_200m = compute_image_footprint_on_surface(camera_x10, 200)
expected_footprint_at_200m = expected_footprint_at_100m * 2

print(f"Footprint at 200m = {footprint_at_200m}")

assert np.allclose(footprint_at_200m, expected_footprint_at_200m, atol=1e-2)

Footprint at 200m = [331.75662 248.9218 ]


## Ground Sampling Distance

Ground sampling distance is the length of the ground (in m) captured by a single pixel. We have the image footpring (the dimensions of ground captured by the whole sensor, and the number of pixels along the horizontal and vertical dimension. Can we get GSD from these two quantities?

In [7]:
gsd_at_100m = compute_ground_sampling_distance(camera_x10, 100)
expected_gsd_at_100m = 0.0202

print(f"GSD at 100m: {gsd_at_100m}")

assert np.allclose(gsd_at_100m, expected_gsd_at_100m, atol=1e-4)

GSD at 100m: 0.020248817279934883


## Bonus: Reprojection from 2D to 3D

If we have a 2d pixel location of a point along with the camera model, can we go back to 3D?
Do we need any additional information.


$\color{red}{\text{TODO: }}$ Implement function `reproject_image_point_to_world` in `src/camera_utils.py` and demonstrate it by running it in the notebook. Confirm that your reprojection + projection function are consistent.

In [8]:
point_3d = np.array([25, -30, 50], dtype=np.float32)
uv = project_world_point_to_image(camera_x10, point_3d)
print(f"{point_3d} projected to {uv}")

reproject_3d = reproject_image_point_to_world(camera_x10, uv, 50)
print(f"reprojected from image point {uv} back to real 3D point {reproject_3d} by giving Z dimention")

assert np.allclose(point_3d, reproject_3d)

[ 25. -30.  50.] projected to [6564.7803   109.60571]
reprojected from image point [6564.7803   109.60571] back to real 3D point [ 25 -30  50] by giving Z dimention


# Week 3: Model the user requirements

For this week, we will model the dataset specifications.

- Overlap: the ratio (in 0 to 1) of scene shared between two consecutive images.
- Sidelap: the ratio (in 0 to 1) of scene shared between two images in adjacent rows.
- Height: the height of the scan above the ground (in meters).
- Scan_dimension_x: the horizontal size of the rectangle to be scanned
- Scan_dimension_y: the vertical size of the rectangle to be scanned
- exposure_time_ms: the exposure time for each image (in milliseconds).


$\color{red}{\text{TODO: }}$ Implement `DatasetSpec` in `src/data_model.py`


In [9]:
# Model the nomimal dataset spec

overlap = 0.7
sidelap = 0.7
height = 30.48 # 100 ft
scan_dimension_x = 150
scan_dimension_y = 150
exposure_time_ms = 2 # 1/500 exposure time

dataset_spec = DatasetSpec(overlap, sidelap, height, scan_dimension_x, scan_dimension_y, exposure_time_ms)

print(f"Nominal specs: {dataset_spec}")

Nominal specs: DatasetSpec(overlap=0.7, sidelap=0.7, height=30.48, scan_dimension_x=150, scan_dimension_y=150, exposure_time_ms=2)


#  Week 4: Compute Distance Between Photos

The overlap and sidelap are the ratio of the dimensions shared between two photos. We already know the footprint of a single image at a given distance. Can we convert the ratio into actual distances? And how does the distance on the surface relate to distance travelled by the camera?

$\color{red}{\text{TODO: }}$ Implement `compute_distance_between_images` in `src/plan_computation.py`



In [10]:
computed_distances = compute_distance_between_images(camera_x10, dataset_spec)
expected_distances = np.array([15.17, 11.38], dtype=np.float32)

print(f"Computed distance for X10 camera with nominal dataset specs: {computed_distances}")

assert np.allclose(computed_distances, expected_distances, atol=1e-2)

Computed distance for X10 camera with nominal dataset specs: [15.167914 11.380706]


$\color{red}{\text{TODO: }}$ define more specifications/camera parameters and check the computed distances. Does that align with your expections


In [11]:
camera_ = copy.copy(camera_x10)
dataset_spec_ = copy.copy(dataset_spec)

# Default
computed_distances_ = compute_distance_between_images(camera_, dataset_spec_)
print(f"Computed distance: {computed_distances_}")



# Height Doubling
dataset_spec_.height = dataset_spec_.height * 2
computed_distances_ = compute_distance_between_images(camera_, dataset_spec_)
print(f"Height Doubled: {computed_distances_}")
# Doubling the height increases the camera's footprint on the ground, 
# so the distance between images also increases to maintain the same overlap and sidelap.



# Reduced Overlap
dataset_spec_ = copy.copy(dataset_spec)
dataset_spec_.overlap = dataset_spec_.overlap / 2
computed_distances_ = compute_distance_between_images(camera_, dataset_spec_)
print(f"Reduced Overlap: {computed_distances_}")
# With less overlap, each image captures more unique ground area, so the distance between images increases.



# Increased Sidelap
dataset_spec_ = copy.copy(dataset_spec)
dataset_spec_.sidelap = 0.9
computed_distances_ = compute_distance_between_images(camera_, dataset_spec_)
print(f"Increased Sidelap: {computed_distances_}")
# With more sidelap, images must be closer together horizontally to achieve the higher side-to-side overlap



# Increase focal length
camera_ = copy.copy(camera_x10)
dataset_spec_ = copy.copy(dataset_spec)
camera_.fx *= 2
computed_distances_ = compute_distance_between_images(camera_, dataset_spec_)
print(f"Increased Focal Length: {computed_distances_}")
# A longer focal length narrows the camera's field of view, reducing the ground footprint, so the distance between images decreases.




# Decrease focal length
camera_ = copy.copy(camera_x10)
dataset_spec_ = copy.copy(dataset_spec)
camera_.fy //= 2
computed_distances_ = compute_distance_between_images(camera_, dataset_spec_)
print(f"Decreased Focal Length: {computed_distances_}")

Computed distance: [15.167914 11.380706]
Height Doubled: [30.335829 22.761412]
Reduced Overlap: [32.86381  11.380706]
Increased Sidelap: [15.167914   3.7935684]
Increased Focal Length: [ 7.583957 11.380706]
Decreased Focal Length: [15.167914 22.76367 ]


## Bonus: Non-Nadir photos

We have solved for the distance assuming that the camera is facing straight down to the ground. This is called [Nadir scanning](https://support.esri.com/en-us/gis-dictionary/nadir). However, in practise we might want a custom gimbal angle.

Your bonus task is to make the distance computation general. Introduce the `camera_angle` parameter in the dataset specification, and work out how to adapt your computation. Feel free to reach out to Ayush to discuss ideas and assumptions!

In [12]:
from src.plan_computation import compute_distance_between_images_method2
def percent_error(a, b):
    x_error = np.abs(a[0] - b[0]) / ((a[0] + b[0]) / 2) * 100
    y_error = np.abs(a[1]-b[1]) / ((a[1] + b[1]) / 2) * 100
    return x_error, y_error

"""
Method1, we account for the tilt by adjusting the effective altitude with height /= np.cos(camera_angle), 
which compensates for the angled view. 
This approach assumes that the camera's footprint on the ground is still a simple projection that scales with altitude.

Method2, we calculate separate top, bottom, left, and right edges based on field of view and trigonometric calculations. 
This approach models the distortion from the angle more accurately by explicitly calculating the distances for each edge. As the angle increases, 
the distortion grows, so this approach may yield significantly larger values at high angles.
"""
# with 0 degree angle
method1 = compute_distance_between_images(camera_x10, dataset_spec, camera_angle=0)
method2 = compute_distance_between_images_method2(camera_x10, dataset_spec, camera_angle=0)
print(f"Computed distance for X10 camera with 0 degree angle:")
print(f"method1: {method1}")
print(f"method2: {method2}")
diff_x, diff_y = percent_error(method1, method2)
print(f"percent errors: {diff_x:.5f}%, {diff_y:.5f}%")
print('-------------------------------------')

# with 20 degree angle
method1 = compute_distance_between_images(camera_x10, dataset_spec, camera_angle=20)
method2 = compute_distance_between_images_method2(camera_x10, dataset_spec, camera_angle=20)
print(f"Computed distance for X10 camera with 20 degree angle:")
print(f"method1: {method1}")
print(f"method2: {method2}")
diff_x, diff_y = percent_error(method1, method2)
print(f"percent errors: {diff_x:.3f}%, {diff_y:.3f}%")
print('-------------------------------------')

# with 30 degree angle
method1 = compute_distance_between_images(camera_x10, dataset_spec, camera_angle=30)
method2 = compute_distance_between_images_method2(camera_x10, dataset_spec, camera_angle=30)
print(f"Computed distance for X10 camera with 30 degree angle:")
print(f"method1: {method1}")
print(f"method2: {method2}")
diff_x, diff_y = percent_error(method1, method2)
print(f"percent errors: {diff_x:.3f}%, {diff_y:.3f}%")
print('-------------------------------------')


# with 45 degree angle
method1 = compute_distance_between_images(camera_x10, dataset_spec, camera_angle=45)
method2 = compute_distance_between_images_method2(camera_x10, dataset_spec, camera_angle=45)
print(f"Computed distance for X10 camera with 45 degree angle:")
print(f"method1: {method1}")
print(f"method2: {method2}")
diff_x, diff_y = percent_error(method1, method2)
print(f"percent errors: {diff_x:.3f}%, {diff_y:.3f}%")


"""
In summary
Method 1 is suitable for small angles
while Method 2 provides more accurate results at larger angles due to its detailed handling of perspective distortion.
"""


Computed distance for X10 camera with 0 degree angle:
method1: [15.167914 11.380706]
method2: [15.1679125 11.380705 ]
percent errors: 0.00001%, 0.00001%
-------------------------------------
Computed distance for X10 camera with 20 degree angle:
method1: [16.141357 12.111094]
method2: [18.899546 13.585316]
percent errors: 15.743%, 11.474%
-------------------------------------
Computed distance for X10 camera with 30 degree angle:
method1: [17.514399 13.141306]
method2: [26.24082  17.423422]
percent errors: 39.887%, 28.020%
-------------------------------------
Computed distance for X10 camera with 45 degree angle:
method1: [21.450668 16.094748]
method2: [97.19606  37.147106]
percent errors: 127.682%, 79.082%


'\nIn summary\nMethod 1 is suitable for small angles\nwhile Method 2 provides more accurate results at larger angles due to its detailed handling of perspective distortion.\n'

# Week 5: Compute Maximum Speed For Blur Free Photos

To restrict motion blur due to camera movement to tolerable limits, we need to restrict the speed such that the image contents move less than 1px away. 

How much does 1px of movement translate to movement of the scene on the ground? It is the ground sampling distance!
From previous week, we know that this is the maximum movement the camera can have. 
We have the distance now. To get speed we need to divide it with time. Do we have time already in our data models?

$\color{red}{\text{TODO: }}$ Implement `compute_speed_during_photo_capture` in `src/plan_computation.py`.

In [13]:
computed_speed = compute_speed_during_photo_capture(camera_x10, dataset_spec, allowed_movement_px=1)
expected_speed = 3.09

print(f"Computed speed during photo captures: {computed_speed:.2f}")

assert np.allclose(computed_speed, expected_speed, atol=1e-2)

Computed speed during photo captures: 3.09


$\color{red}{\text{TODO: }}$ define more specifications/camera parameters and check the computed distances. Does that align with your expections


In [14]:
camera_ = copy.copy(camera_x10)
dataset_spec_ = copy.copy(dataset_spec)

computed_speed_ = compute_speed_during_photo_capture(camera_, dataset_spec_)
print(f"Max speed: {computed_speed_:.2f}")
print('------------------------------')


# Height
camera_ = copy.copy(camera_x10)
dataset_spec_ = copy.copy(dataset_spec)
dataset_spec_.height *= 2
computed_speed_ = compute_speed_during_photo_capture(camera_, dataset_spec_)
print('Double the height')
print(f"Max speed: {computed_speed_:.2f}")
"""
A higher altitude results in a larger ground sampling distance (GSD), meaning each pixel represents a larger ground area. 
As a result, the drone can move faster while still keeping within the 1-pixel motion blur threshold
"""
print('------------------------------')


# Exposure Time
camera_ = copy.copy(camera_x10)
dataset_spec_ = copy.copy(dataset_spec)
dataset_spec_.exposure_time_ms *= 2
computed_speed_ = compute_speed_during_photo_capture(camera_, dataset_spec_)
print('Longer Exposure Time')
print(f"Max speed: {computed_speed_:.2f}")
"""
A longer exposure time means the camera sensor is active for a longer period, increasing the risk of motion blur. 
To prevent exceeding the 1-pixel movement limit, the drone must move slower
"""
print('------------------------------')


# Focal Length
camera_ = copy.copy(camera_x10)
dataset_spec_ = copy.copy(dataset_spec)
camera_.fx *= 2
computed_speed_ = compute_speed_during_photo_capture(camera_, dataset_spec_)
print('Longer Focal Length')
print(f"Max speed: {computed_speed_:.2f}")
"""
#A longer exposure time means the camera sensor is active for a longer period, increasing the risk of motion blur. 
#To prevent exceeding the 1-pixel movement limit, the drone must move slower
"""
print('------------------------------')

Max speed: 3.09
------------------------------
Double the height
Max speed: 6.17
------------------------------
Longer Exposure Time
Max speed: 1.54
------------------------------
Longer Focal Length
Max speed: 1.54
------------------------------


# Week 6: Generate Full Flight Plans  

We now have all the tools to generate the full flight plan.

Steps for this week:
1. Define the `Waypoint` data model. What attributes should the data model have?
   1. For Nadir scans, just the position of the camera is enough as we will always look drown to the ground.
   2. For general case (bonus), we also need to define where the drone will look at@
3. Implement the function `generate_photo_plan_on_grid` to generate the full plan.
   1. Compute the maximum distance between two images, horizontally and vertically.
   2. Layer the images such that we cover the whole scan area. Note that you need to take care when the scan dimension is not a multiple of distance between images. Example: to cover 45m length with 10m between images, we would need 4.5 images. Not possible. 4 images would not satisfy the overlap, so we should go with 5. How should we arrange 5 images in the given 45m.
   3. Assign the speed to each waypoint.

$\color{red}{\text{TODO: }}$ Implement:
- `Waypoint` in `src/data_model.py`
- `generate_photo_plan_on_grid` in `src/plan_computation.py`.

In [15]:
computed_plan = generate_photo_plan_on_grid(camera_x10, dataset_spec) 

print(f"Computed plan with {len(computed_plan)} waypoints")

Computed plan with 140 waypoints


In [23]:
MAX_NUM_WAYPOINTS_TO_PRINT = 20

for idx, waypoint in enumerate(computed_plan[:20]):
    print(f"Idx {idx}: {waypoint}")
if len(computed_plan) >= MAX_NUM_WAYPOINTS_TO_PRINT:
    print("...")

Idx 0: Waypoint(X=np.float32(0.0), Y=np.float32(0.0), Z=60.96, speed=np.float32(6.1718397))
Idx 1: Waypoint(X=np.float32(30.335829), Y=np.float32(0.0), Z=60.96, speed=np.float32(6.1718397))
Idx 2: Waypoint(X=np.float32(60.671658), Y=np.float32(0.0), Z=60.96, speed=np.float32(6.1718397))
Idx 3: Waypoint(X=np.float32(91.007484), Y=np.float32(0.0), Z=60.96, speed=np.float32(6.1718397))
Idx 4: Waypoint(X=np.float32(121.343315), Y=np.float32(0.0), Z=60.96, speed=np.float32(6.1718397))
Idx 5: Waypoint(X=np.float32(121.343315), Y=np.float32(22.761412), Z=60.96, speed=np.float32(6.1718397))
Idx 6: Waypoint(X=np.float32(91.007484), Y=np.float32(22.761412), Z=60.96, speed=np.float32(6.1718397))
Idx 7: Waypoint(X=np.float32(60.671658), Y=np.float32(22.761412), Z=60.96, speed=np.float32(6.1718397))
Idx 8: Waypoint(X=np.float32(30.335829), Y=np.float32(22.761412), Z=60.96, speed=np.float32(6.1718397))
Idx 9: Waypoint(X=np.float32(0.0), Y=np.float32(22.761412), Z=60.96, speed=np.float32(6.1718397))


## Bonus: Time computation 

if you have some time, you can implement a time computation function. We can make the drone fly as fast as possible between photos, but make sure it can decelerate back to the required speed at the photos. Please use the following data: 
- Max drone speed: 16m/s.
- Max acceleration: 3.5 m/s^2.

Hint: you might need to use a trapezoidal speed profile

In [17]:
from src.plan_computation import approximate_time

max_speed = 16
a_max = 3.5
finish_time, achieve_speed = approximate_time(camera_x10, dataset_spec, computed_plan, max_speed, a_max)
print(f'This plan will take {len(computed_plan)} photos and finish in {int(finish_time)} seconds')


max_speed = 32
a_max = 3.5
time = approximate_time(camera_x10, dataset_spec, computed_plan, max_speed, a_max)[0]
print(f'This plan will take {len(computed_plan)} photos and finish in {int(time)} seconds')

# For above 2 test cases, even if double the max speed for the drone,
# it will take the same amount of time to complete the plan.
# So, I may have assume that the distance between 2 photos is not enough for the drone to fly with a higher speed



# trying with different overlaps
max_speed = 16
a_max = 3.5
dataset_copy = copy.copy(dataset_spec)
dataset_copy.overlap = 0.9
new_plan = generate_photo_plan_on_grid(camera_x10, dataset_copy)
time = approximate_time(camera_x10, dataset_copy, new_plan, max_speed, a_max)[0]
print(f'This plan will take {len(new_plan)} photos and finish in {int(time)} seconds')




This plan will take 140 photos and finish in 383 seconds
This plan will take 140 photos and finish in 383 seconds
This plan will take 420 photos and finish in 511 seconds


# Week 7: Visualize Flight Plans

This week, we will use a third party plotting framework called [Plotly](https://plotly.com/python/) to visualize our plans. Please follow this [tutorial](https://www.kaggle.com/code/kanncaa1/plotly-tutorial-for-beginners) to gain some basic experience with Plotly, and then come up with your own visualization function. You are free to choose to come up with your own visualization, and use something other than Plotly.

$\color{red}{\text{TODO: }}$ Implement `plot_photo_plan` in `src/visualization.py`

In [18]:
fig = plot_photo_plan(computed_plan, achieve_speed, finish_time)
fig.show()

$\color{red}{\text{TODO: }}$ Compute the following ablations (and any other you can think of)

1. Change overlap and confirm it affects the consecutive images
2. Change sidelap and confirm it does not affect the consecutive images
3. Change the height of the scan and document the affect on scan plans
4. Change exposure time

In [19]:
camera_ = copy.deepcopy(camera_x10)
dataset_spec_ = copy.deepcopy(dataset_spec)
print(camera_x10)
print(dataset_spec)

# change the exposure time
dataset_spec_.exposure_time_ms = 1000

max_speed = 16
a_max = 3.5
computed_plan = generate_photo_plan_on_grid(camera_, dataset_spec_)
finish_time, achieve_speed = approximate_time(camera_, dataset_spec_, computed_plan, max_speed, a_max)

fig = plot_photo_plan(computed_plan, achieve_speed, finish_time)
fig.show()

Camera(fx=4938.56, fy=4936.49, cx=4095.5, cy=3071.5, sensor_size_x_mm=13.107, sensor_size_y_mm=9.83, image_size_x_px=8192, image_size_y_px=6144)
DatasetSpec(overlap=0.7, sidelap=0.7, height=30.48, scan_dimension_x=150, scan_dimension_y=150, exposure_time_ms=2)


In [20]:
camera_ = copy.deepcopy(camera_x10)
dataset_spec_ = copy.deepcopy(dataset_spec)
print(camera_x10)
print(dataset_spec)

# overlap
dataset_spec_.overlap = 0.9

max_speed = 16
a_max = 3.5
computed_plan = generate_photo_plan_on_grid(camera_, dataset_spec_)
finish_time, achieve_speed = approximate_time(camera_, dataset_spec_, computed_plan, max_speed, a_max)

fig = plot_photo_plan(computed_plan, achieve_speed, finish_time)
fig.show()

Camera(fx=4938.56, fy=4936.49, cx=4095.5, cy=3071.5, sensor_size_x_mm=13.107, sensor_size_y_mm=9.83, image_size_x_px=8192, image_size_y_px=6144)
DatasetSpec(overlap=0.7, sidelap=0.7, height=30.48, scan_dimension_x=150, scan_dimension_y=150, exposure_time_ms=2)


In [21]:
camera_ = copy.deepcopy(camera_x10)
dataset_spec_ = copy.deepcopy(dataset_spec)
print(camera_x10)
print(dataset_spec)

# less sidelap
dataset_spec_.sidelap = 0.1

max_speed = 16
a_max = 3.5
computed_plan = generate_photo_plan_on_grid(camera_, dataset_spec_)
finish_time, achieve_speed = approximate_time(camera_, dataset_spec_, computed_plan, max_speed, a_max)

fig = plot_photo_plan(computed_plan, achieve_speed, finish_time)
fig.show()

Camera(fx=4938.56, fy=4936.49, cx=4095.5, cy=3071.5, sensor_size_x_mm=13.107, sensor_size_y_mm=9.83, image_size_x_px=8192, image_size_y_px=6144)
DatasetSpec(overlap=0.7, sidelap=0.7, height=30.48, scan_dimension_x=150, scan_dimension_y=150, exposure_time_ms=2)


In [22]:
camera_ = copy.deepcopy(camera_x10)
dataset_spec_ = copy.deepcopy(dataset_spec)
print(camera_x10)
print(dataset_spec)

# height
dataset_spec_.height *= 2

max_speed = 16
a_max = 3.5
computed_plan = generate_photo_plan_on_grid(camera_, dataset_spec_)
finish_time, achieve_speed = approximate_time(camera_, dataset_spec_, computed_plan, max_speed, a_max)

fig = plot_photo_plan(computed_plan, achieve_speed, finish_time)
fig.show()

Camera(fx=4938.56, fy=4936.49, cx=4095.5, cy=3071.5, sensor_size_x_mm=13.107, sensor_size_y_mm=9.83, image_size_x_px=8192, image_size_y_px=6144)
DatasetSpec(overlap=0.7, sidelap=0.7, height=30.48, scan_dimension_x=150, scan_dimension_y=150, exposure_time_ms=2)


# Meta-Learning And Summary

## 1. Comprehensive Camera System Modeling
- Developed camera's intrinsic and extrinsic parameters using `dataclasses` in Python
- Applied the **pinhole camera model** for accurate projection of 3D world points into 2D image coordinates and vice versa (reprojection).

---

## 2. Ground Sampling Distance (GSD) and Image Footprint Analysis
- The concept of **Ground Sampling Distance (GSD)** to quantify the real-world size captured by a single pixel.
- Calculated **image footprints** at varying altitudes, understanding:
  - How the camera’s field of view changes with height
  - The impact on flight planning

---
## 3. Speed Optimization to Prevent Motion Blur
- Modeled maximum drone speed to prevent motion blur using GSD and exposure time constraints.
- Analyzed the influence of flight parameters (e.g., altitude, focal length, exposure time) on motion blur and performance.

---

## 4. Drone Trajectory Planning
- Designed efficient **lawnmower trajectory patterns**, balancing:
  - Complete ground coverage
  - Optimized flight efficiency
- Incorporated overlap and sidelap parameters to ensure data continuity.


---

## 5. Visualization and Validation with Plotly
- Designed interactive flight path visualizations using **Plotly**, showcasing:
  - Waypoints
  - Speed
  - Timing data
- Validated trajectory assumptions (e.g., impact of overlap, sidelap, height) through detailed visual analysis.


---

## 6. General Skills and Insights
- Connected theoretical principles with practical drone applications, addressing:
  - Pinhole Camera
  - Motion blur
  - Scan completeness
  - Efficiency
- Strengthened analytical thinking through systematic validation and sensitivity analyses.

